In [ ]:
'''
Done: whole corpus parsed (only 'text' and 'conclusion' are saved') to
the single .csv file. Whole corpus is then indexed and 
evaluated via 'map' and 'ndcg'.
'''

In [6]:
!pip install ijson

     |████████████████████████████████| 126 kB 15.7 MB/s 


In [15]:
import ijson
import re
data_set = set()
id_text = {}
filenames = ['debateorg', 'debatepedia', 'debatewise', 'idebate', 'parliamentary']
for filename in filenames:
  path = '/content/drive/MyDrive/Bachelor_webis/' + filename + '.json'
  with open(path, 'r', encoding='utf-8') as f:
      objects = ijson.items(f, 'arguments.item')
      for i, o in enumerate(objects):
        # text = 
        id_text[i] = o['id'], o["premises"][0]["text"] + o["conclusion"] 

In [16]:
import pandas as pd
df = pd.DataFrame.from_dict(id_text).transpose()
df = df.rename(columns={0: "docno", 1: "text"})


In [17]:
df.to_csv(index=False, path_or_buf='/content/drive/MyDrive/workspace/corpus.csv')

In [ ]:
!pip install python-terrier
import pyterrier as pt
if not pt.started():
  pt.init()

In [26]:
pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(df["text"], df["docno"])

13:46:17.839 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (Sdbc891ec-A55b98352) - further warnings are suppressed
13:50:40.562 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 57 empty documents
13:50:40.918 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key Saeff5ad8-Afba459ff is not unique: 160773,160770
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [20]:
from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)


In [21]:
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

In [27]:
index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)
qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

In [28]:
res

,qid,docid,docno,rank,score,query
0,51,297468,S885c6b4f-Ad3259e81,0,15.174829,do we need sex education in schools
1,51,330918,S6353b9ac-A9cb2062c,1,14.984042,do we need sex education in schools
2,51,222529,S428a8bea-Aba4c7b80,2,14.776028,do we need sex education in schools
3,51,279241,Sd32774d-A5bbc95,3,14.715350,do we need sex education in schools
4,51,330916,S6353b9ac-A2df6348e,4,14.638091,do we need sex education in schools
...,...,...,...,...,...,...
49995,100,256729,S2ea519c-Aed35ffc8,995,3.581751,do we need cash
49996,100,29220,Sa68e9864-Af79ef325,996,3.580354,do we need cash
49997,100,231260,Sfb30ce24-Aa94950e7,997,3.580354,do we need cash
49998,100,294707,S78819fc5-Ae076e471,998,3.576581,do we need cash


In [57]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5"])
eval

{'ndcg_cut_5': 0.5527540851759042}